In [1]:
import sqlite3
import json
from datetime import datetime
import pandas as pd

In [2]:
dbname = '2017-12'
path_to_RC_file = 'RC_'+ dbname  # ie RC_2005-12

In [3]:
# Instantiate the database
# the instance is c, and used globally in transaction blder 

sql_transaction = []

connection = sqlite3.connect(dbname+'.db')
c = connection.cursor()

def create_table():
    c.execute("CREATE TABLE IF NOT EXISTS parent_reply(parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)")

def format_data(data):
    data = data.replace('\n',' newlinechar ').replace('\r',' newlinechar ').replace('"',"'")
    return data

def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
        connection.commit()
        sql_transaction = []

def sql_insert_replace_comment(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?, unix = ?, score = ? WHERE parent_id =?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

def acceptable(data):
    if len(data.split(' ')) > 1000 or len(data) < 1:
        return False
    elif len(data) > 32000:
        return False
    elif data == '[deleted]':
        return False
    elif data == '[removed]':
        return False
    else:
        return True

def find_parent(pid):
    try:
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False

def find_existing_score(pid):
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False

In [ ]:
# The data from the RC file will be added to the database 

create_table()
row_counter = 0
paired_rows = 0
start_row =  3600000
cleanup = 1000000
output_every = 100000
score_thres = 0

with open(path_to_RC_file, buffering=1000) as f:
    for row in f:
        #print(row)
        #time.sleep(555)
        row_counter += 1

        if row_counter > start_row:
            try:
                row = json.loads(row)
                parent_id = row['parent_id'].split('_')[1]
                body = format_data(row['body'])
                created_utc = row['created_utc']
                score = row['score']
                comment_id = row['id']

                subreddit = row['subreddit']
                parent_data = find_parent(parent_id)


                existing_comment_score = find_existing_score(parent_id)
                if existing_comment_score:
                    if score > existing_comment_score:
                        if acceptable(body):
                            sql_insert_replace_comment(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)

                else:
                    if acceptable(body):
                        if parent_data:
                            #print(score)
                            if score >= score_thres:
                                #print(parent_id, comment_id, parent_data, body)
                                sql_insert_has_parent(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                                paired_rows += 1
                        else:
                            sql_insert_no_parent(comment_id,parent_id,body,subreddit,created_utc,score)
            except Exception as e:
                print(str(e))

        if row_counter % output_every == 0:
            print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_counter, paired_rows, str(datetime.now())))

        if row_counter > start_row:
            if row_counter % cleanup == 0:
                print("Cleanin up!")
                sql = "DELETE FROM parent_reply WHERE parent IS NULL"
                c.execute(sql)
                connection.commit()
                c.execute("VACUUM")
                connection.commit()
                
print("Total Rows Scanned: ", row_counter )

Total Rows Read: 100000, Paired Rows: 0, Time: 2018-02-04 14:25:47.287006
Total Rows Read: 200000, Paired Rows: 0, Time: 2018-02-04 14:25:47.827375
Total Rows Read: 300000, Paired Rows: 0, Time: 2018-02-04 14:25:48.375749
Total Rows Read: 400000, Paired Rows: 0, Time: 2018-02-04 14:25:48.921506
Total Rows Read: 500000, Paired Rows: 0, Time: 2018-02-04 14:25:49.476219
Total Rows Read: 600000, Paired Rows: 0, Time: 2018-02-04 14:25:50.027364
Total Rows Read: 700000, Paired Rows: 0, Time: 2018-02-04 14:25:50.572958
Total Rows Read: 800000, Paired Rows: 0, Time: 2018-02-04 14:25:51.117431
Total Rows Read: 900000, Paired Rows: 0, Time: 2018-02-04 14:25:51.677339
Total Rows Read: 1000000, Paired Rows: 0, Time: 2018-02-04 14:25:52.229112
Total Rows Read: 1100000, Paired Rows: 0, Time: 2018-02-04 14:25:52.801076
Total Rows Read: 1200000, Paired Rows: 0, Time: 2018-02-04 14:25:53.341097
Total Rows Read: 1300000, Paired Rows: 0, Time: 2018-02-04 14:25:53.907471
Total Rows Read: 1400000, Paired R

In [51]:
#create_table()
#sql_insert_has_parent(1,2,"hello","Hi","reddit",0,1)
#sql_insert_no_parent(1,2,"Hi","reddit",0,1)
df = pd.read_sql_query("SELECT * FROM  parent_reply ", connection)
df

,parent_id,comment_id,parent,comment,subreddit,unix,score
0,7gm42l,dql1dzj,None,Always inviting you to go out to eat. My spoil...,AskReddit,1512086400,1
1,7giy5a,dql1dzk,None,Why do you not have image previews on your pag...,HoneySelect,1512086400,1
2,7gquvj,dql1dzl,None,All my buddies love Pacific Rim and always tal...,AskReddit,1512086400,179
3,7gjjm0,dql1dzm,None,"sounds like coil whine , when playing cities s...",pcmasterrace,1512086400,1
4,dql0d4o,dql1dzn,None,&lt;3,freefolk,1512086400,2
5,7gmrzf,dql1dzo,None,I'm just wondering what needs conveying with t...,Justrolledintotheshop,1512086400,3
6,7gqpou,dql1dzp,None,Look at that schnozz,milliondollarextreme,1512086400,5
7,7gpgpp,dql1dzq,None,It might take stray cats.,pics,1512086400,3
8,dql0sks,dql1dzr,None,Smart,Brawlstars,1512086400,1
9,dqk36z9,dql1dzs,None,I can't even use this sub anymore because when...,Xenoblade_Chronicles,1512086400,2


In [53]:
# Make training Data

dbnames = ['2017-12']

for dbname in dbnames:
    connection = sqlite3.connect(dbname+'.db')
    c = connection.cursor()
    limit = 5000
    last_unix = 0
    cur_length = limit
    counter = 0
    test_done = False

    while cur_length == limit:

        df = pd.read_sql("SELECT * FROM parent_reply WHERE unix > {} and parent NOT NULL and score > 0 ORDER BY unix ASC LIMIT {}".format(last_unix,limit),connection)
        last_unix = df.tail(1)['unix'].values[0]
        cur_length = len(df)

        if not test_done:
            with open('test.from','a', encoding='utf8') as f:
                for content in df['parent'].values:
                    f.write(content+'\n')

            with open('test.to','a', encoding='utf8') as f:
                for content in df['comment'].values:
                    f.write(str(content)+'\n')

            test_done = True

        else:
            with open('train.from','a', encoding='utf8') as f:
                for content in df['parent'].values:
                    f.write(content+'\n')

            with open('train.to','a', encoding='utf8') as f:
                for content in df['comment'].values:
                    f.write(str(content)+'\n')

        counter += 1
        if counter % 20 == 0:
            print(counter*limit,'rows completed so far')

100000 rows completed so far
200000 rows completed so far
300000 rows completed so far
400000 rows completed so far
500000 rows completed so far
600000 rows completed so far
